In [1]:
MAIN_PATH = "/home/carlos/MasterDS/tfm"
JSON_DATA_PATH = '{}/data/json/'.format(MAIN_PATH)
CSV_DATA_PATH = '{}/data/csv/'.format(MAIN_PATH)

In [2]:
import sys
sys.path.insert(0, MAIN_PATH)

In [13]:
%load_ext autoreload
%autoreload 2
from scripts.text.article_text_processor import ArticleTextProcessor
from scripts.text.basic_text_processor import BasicTextProcessor
from scripts.extractive_summary.ltr.learn_to_rank import LearnToRank

from scripts.conf import TEAMS

from rouge import Rouge

%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Learn to Rank

In [4]:
processor = ArticleTextProcessor()
text_proc = BasicTextProcessor()

In [5]:
all_files = processor.load_json()

In [6]:
season_file = 'premier_league_2019_2020.json'
league_season_teams = TEAMS[season_file.split('.')[0]]

In [7]:
match_dict = all_files[season_file]['https://www.bbc.com/sport/football/49791610']
events = match_dict['events']

## ROUGE

Utilizaremos esta métrica para asignar un score a cada par evento-frase artículo, para generar un target que indique qué evento tiene 
más opciones de aparecer en el resumen. Con este target se entrenará un modelo Learn to rank, de tal forma que se pueda construir un resumen 
con el conjunto de eventos más representativo de cada partido.

In [61]:
ltr = LearnToRank()

Setting target metric to rouge


In [62]:
event_article_list = ltr.create_match_targets(match_dict, verbose=False)

In [138]:
event_article_list

[{'event_ix': 0, 'sentence_ix': 0, 'score': 0.0},
 {'event_ix': 1, 'sentence_ix': 0, 'score': 0.0},
 {'event_ix': 2, 'sentence_ix': 13, 'score': 0.037037037037037035},
 {'event_ix': 3, 'sentence_ix': 32, 'score': 0.07142857142857142},
 {'event_ix': 4, 'sentence_ix': 32, 'score': 0.07142857142857142},
 {'event_ix': 5, 'sentence_ix': 0, 'score': 0.0},
 {'event_ix': 6, 'sentence_ix': 0, 'score': 0.0},
 {'event_ix': 7, 'sentence_ix': 0, 'score': 0.0},
 {'event_ix': 8, 'sentence_ix': 23, 'score': 0.05},
 {'event_ix': 9, 'sentence_ix': 0, 'score': 0.0},
 {'event_ix': 10, 'sentence_ix': 20, 'score': 0.0625},
 {'event_ix': 11, 'sentence_ix': 13, 'score': 0.037037037037037035},
 {'event_ix': 12, 'sentence_ix': 23, 'score': 0.05},
 {'event_ix': 13, 'sentence_ix': 0, 'score': 0.0},
 {'event_ix': 14, 'sentence_ix': 4, 'score': 0.034482758620689655},
 {'event_ix': 15, 'sentence_ix': 20, 'score': 0.0625},
 {'event_ix': 16, 'sentence_ix': 0, 'score': 0.0},
 {'event_ix': 17, 'sentence_ix': 32, 'score'

Al usar esta métrica, las correspondencias que se obtienen no tienen demasiado sentido...

In [63]:
ltr.print_scores_info(match_dict, event_article_list)

Score: 0.2926829219750149
Event: Attempt missed. Youri Tielemans (Leicester City) right footed shot from outside the box is too high. Assisted by Wilfred Ndidi with a headed pass.
Nearest article sentence: On a shocking day for Newcastle, Wilfred Ndidi added a late fifth goal with a shot on the turn.

Score: 0.275862064684899
Event: Isaac Hayden (Newcastle United) is shown the red card.
Nearest article sentence: Newcastle boss Steve Bruce on Isaac Hayden's red card: "I've got no complaints, the red card was a poor challenge. 

Score: 0.2758620641141499
Event: Wilfred Ndidi (Leicester City) wins a free kick in the attacking half.
Nearest article sentence: On a shocking day for Newcastle, Wilfred Ndidi added a late fifth goal with a shot on the turn.

Score: 0.2758620641141499
Event: Wilfred Ndidi (Leicester City) wins a free kick in the defensive half.
Nearest article sentence: On a shocking day for Newcastle, Wilfred Ndidi added a late fifth goal with a shot on the turn.

Score: 0.2448

## Distancia coseno

In [14]:
count_vec_kwargs = {'ngram_range': (1, 2), 'strip_accents': 'unicode'}

In [15]:
ltr = LearnToRank(target_metric = 'cosine_tfidf', drop_teams=True)

Setting target metric to cosine_tfidf


In [16]:
event_article_list = ltr.create_match_targets(match_dict, verbose=True, league_season_team=league_season_teams, **count_vec_kwargs)

Event: Lineups are announced and players are warming up.
Processed event: lineups announced warming
Nearest article sentence: Manchester United and Arsenal played out a grim stalemate at Old Trafford that provided compelling evidence to illustrate just how far away from a Premier League challenge both clubs are.
Processed article sentence: grim stalemate old trafford provided evidence illustrate far away league challenge clubs

Event: First Half begins.
Processed event: half begins
Nearest article sentence: "We didn't get out of the blocks quick enough at the start of the second half.
Processed article sentence: blocks quick start second half

Event: Foul by Matteo Guendouzi (Arsenal).
Processed event: foul matteo guendouzi
Nearest article sentence: United had the edge on chances, although they were grateful to David de Gea for a fine double save from Saka and Matteo Guendouzi in the first half
Processed article sentence: edge chances grateful david gea fine double save saka matteo gue

In [11]:
ltr.print_scores_info(match_dict, event_article_list)

Score: 1.0000000000000002
Event: Substitution, Arsenal. Joe Willock replaces Bukayo Saka.
Nearest article sentence: Bukayo [Saka] is progressing well

Score: 0.8342610459790887
Event: Foul by Bukayo Saka (Arsenal).
Nearest article sentence: Bukayo [Saka] is progressing well

Score: 0.8342610459790887
Event: Hand ball by Bukayo Saka (Arsenal).
Nearest article sentence: Bukayo [Saka] is progressing well

Score: 0.7682692199046739
Event: Bukayo Saka (Arsenal) wins a free kick in the attacking half.
Nearest article sentence: Bukayo [Saka] is progressing well

Score: 0.7682692199046739
Event: Bukayo Saka (Arsenal) wins a free kick in the attacking half.
Nearest article sentence: Bukayo [Saka] is progressing well

Score: 0.7305698634466145
Event: Bukayo Saka (Arsenal) wins a free kick on the left wing.
Nearest article sentence: Bukayo [Saka] is progressing well

Score: 0.6578950908923038
Event: Attempt blocked. Bukayo Saka (Arsenal) right footed shot from the centre of the box is blocked. As

__Examinando tfidf__

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline
import pandas as pd

In [56]:
proc_events, proc_article_sents = ltr._process_events_article(match_dict)

In [57]:
proc_events

['lineups announced warming',
 'half begins',
 'foul matteo guendouzi',
 'scott mctominay wins kick defensive half',
 'granit xhaka wins kick defensive half',
 'foul jesse lingard',
 'corner conceded axel tuanzebe',
 'hand ball bukayo saka',
 'corner conceded calum chambers',
 'foul calum chambers',
 'daniel james wins kick left wing',
 'calum chambers shown yellow card bad foul',
 'offside ashley young tries ball harry offside',
 'hand ball',
 'offside bernd leno tries ball bukayo saka offside',
 'emerick aubameyang wins kick left wing',
 'foul pogba',
 'bukayo saka wins kick attacking half',
 'foul jesse lingard',
 'matteo guendouzi wins kick defensive half',
 'foul',
 'bukayo saka wins kick left wing',
 'foul ashley young',
 'sokratis wins kick defensive half',
 'foul marcus rashford',
 'marcus rashford shown yellow card bad foul',
 'foul lucas',
 'axel tuanzebe wins kick defensive half',
 'attempt saved left footed shot box saved corner',
 'attempt missed pepe left footed shot outs

In [58]:
proc_article_sents

['grim stalemate old trafford provided evidence illustrate far away league challenge clubs',
 'disappointing encounter shadow mighty years gone scott mctominay gave lead end attritional half rising drive edge',
 'equalised hour courtesy emerick aubameyang cool finish',
 'originally ruled offside video assistant confirmed striker onside harry',
 'goalkeeper bernd leno excelled fine saves marcus rashford kick bukayo saka goalbound shot crucially struck victor lindelof flew',
 'mctominay headed chance ashley young corner force winner',
 'takes',
 'means worst start seven games years failing double time season finished',
 'circumstances clubs instantly demonstrated captains led sides',
 'captain old ashley young defensive service aaron bissaka absence injury led granit xhaka newly appointed divisive supporters',
 'young led fashion contesting decisions showing knack getting kevin friend instant numerous occasions',
 'xhaka night best contribution goal getting away foul build appearing evas

In [19]:
len(proc_article_sents)

37

In [20]:
len(proc_events)

125

In [30]:
pipe = Pipeline([('count', CountVectorizer(**count_vec_kwargs)),
                         ('tfid', TfidfTransformer())])

In [31]:
X = pipe.fit_transform(proc_article_sents)

In [187]:
X

<37x521 sparse matrix of type '<class 'numpy.float64'>'
	with 618 stored elements in Compressed Sparse Row format>

In [32]:
tfidf_df = pd.DataFrame(X.todense(), columns=pipe['count'].get_feature_names())

In [13]:
article_sentences = ltr.text_proc.get_sentences(match_dict['article'])
article_sentences_text = [str(sent).replace('\n', '') for sent in article_sentences]

In [35]:
article_sentences_text[1]

'In a disappointing encounter that was a pale shadow of their mighty clashes of years gone by, Scott McTominay gave Manchester United the lead at the end of an attritional first-half with a rising drive from the edge of the area.'

In [36]:
proc_article_sents[1]

'disappointing encounter shadow mighty years gone scott mctominay gave manchester united lead end attritional half rising drive edge'

In [28]:
match_dict['events'][85]

'Foul by Harry Maguire (Manchester United).'

In [29]:
proc_events[85]

'foul harry manchester united'

In [37]:
pd_df_sent = tfidf_df.loc[1]

In [38]:
pd_df_sent[pd_df_sent>0].sort_values(ascending=False)

years gone                 0.177416
lead end                   0.177416
attritional half           0.177416
disappointing              0.177416
disappointing encounter    0.177416
drive edge                 0.177416
encounter                  0.177416
encounter shadow           0.177416
end attritional            0.177416
gave                       0.177416
gave manchester            0.177416
gone                       0.177416
gone scott                 0.177416
half rising                0.177416
lead                       0.177416
attritional                0.177416
shadow                     0.177416
united lead                0.177416
shadow mighty              0.177416
mctominay gave             0.177416
mighty                     0.177416
mighty years               0.177416
rising                     0.177416
rising drive               0.177416
scott                      0.177416
scott mctominay            0.177416
years                      0.159178
drive                      0

In [54]:
events[7]

'Hand ball by Bukayo Saka (Arsenal).'

In [55]:
proc_events[7]

'hand ball bukayo saka arsenal'

In [49]:
X_events = pipe.transform(proc_events)

In [50]:
tfidf_events_df = pd.DataFrame(X_events.todense(), columns=pipe['count'].get_feature_names())

In [52]:
pd_df_sent_event = tfidf_events_df.loc[7]

In [53]:
pd_df_sent_event[pd_df_sent_event>0].sort_values(ascending=False)

ball           0.526185
bukayo saka    0.472097
bukayo         0.472097
saka           0.433720
arsenal        0.298773
Name: 7, dtype: float64

# WMD

WMD (Word Movers Distance) es una distancia basada en la representación de palabras usando Word Embeddings. La principal ventaja de usar word embeddings
es que la distancia puede ser pequeña, aunque no haya palabras en común (sinónimos)

In [13]:
ltr = LearnToRank(target_metric = 'wmd')

Setting target metric to wmd


In [9]:
event_article_list = ltr.create_match_targets(match_dict, verbose=False, league_season_team=league_season_teams)

In [15]:
ltr.print_scores_info(match_dict, event_article_list, reverse=False)

Score: 0.14895175432223806
Event: Goal!  Manchester United 1, Arsenal 1. Pierre-Emerick Aubameyang (Arsenal) left footed shot from the centre of the box to the centre of the goal. Assisted by Bukayo Saka with a through ball.Goal awarded following VAR Review.
Nearest article sentence: Goalkeeper Bernd Leno excelled for Arsenal with fine saves from Maguire and Marcus Rashford's late free-kick, while Bukayo Saka's goalbound shot crucially struck Victor Lindelof and flew over the top.

Score: 0.1491208779912439
Event: Substitution, Manchester United. Fred replaces Jesse Lingard.
Nearest article sentence: signing Nicholas Pepe failed to last the course, being substituted for Reiss Nelson after 72 minutes.

Score: 0.15066254246857064
Event: Attempt missed. Scott McTominay (Manchester United) header from very close range is just a bit too high. Assisted by Ashley Young with a cross following a corner.
Nearest article sentence: In a disappointing encounter that was a pale shadow of their might

In [4]:
import gensim.downloader as api

In [10]:
api.info()['models'].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

In [13]:
api.info()['models']['word2vec-google-news-300']

{'num_records': 3000000,
 'file_size': 1743563840,
 'base_dataset': 'Google News (about 100 billion words)',
 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/word2vec-google-news-300/__init__.py',
 'license': 'not found',
 'parameters': {'dimension': 300},
 'description': "Pre-trained vectors trained on a part of the Google News dataset (about 100 billion words). The model contains 300-dimensional vectors for 3 million words and phrases. The phrases were obtained using a simple data-driven approach described in 'Distributed Representations of Words and Phrases and their Compositionality' (https://code.google.com/archive/p/word2vec/).",
 'read_more': ['https://code.google.com/archive/p/word2vec/',
  'https://arxiv.org/abs/1301.3781',
  'https://arxiv.org/abs/1310.4546',
  'https://www.microsoft.com/en-us/research/publication/linguistic-regularities-in-continuous-space-word-representations/?from=http%3A%2F%2Fresearch.microsoft.com%2Fpubs%2F189726%2Frvec

In [14]:
model = api.load('word2vec-google-news-300')
model.init_sims(replace=True)

In [27]:
event = proc_events[2]

In [28]:
event

'foul matteo guendouzi arsenal'

In [29]:
ref_sent = proc_article_sents[1]

In [18]:
ref_sent

'disappointing encounter shadow mighty years gone scott mctominay gave manchester united lead end attritional half rising drive edge'

In [30]:
model.wmdistance(event, ref_sent)

0.3103198283671672

In [ ]:
a

In [14]:
proc_events

['lineups announced warming',
 'half begins',
 'foul matteo guendouzi arsenal',
 'scott mctominay manchester united wins kick defensive half',
 'granit xhaka arsenal wins kick defensive half',
 'foul jesse lingard manchester united',
 'corner arsenal conceded axel tuanzebe',
 'hand ball bukayo saka arsenal',
 'corner manchester united conceded calum chambers',
 'foul calum chambers arsenal',
 'daniel james manchester united wins kick left wing',
 'calum chambers arsenal shown yellow card bad foul',
 'offside manchester united ashley young tries ball harry offside',
 'hand ball manchester united',
 'offside arsenal bernd leno tries ball bukayo saka offside',
 'emerick aubameyang arsenal wins kick left wing',
 'foul pogba manchester united',
 'bukayo saka arsenal wins kick attacking half',
 'foul jesse lingard manchester united',
 'matteo guendouzi arsenal wins kick defensive half',
 'foul manchester united',
 'bukayo saka arsenal wins kick left wing',
 'foul ashley young manchester unit